`Задание`
`Основные цели этого задания:`

- Попрактиковаться в борьбе с дисбалансом классов

- Научиться заполнять пропуски в данных

- Научиться использовать категориальные признаки.

`Задача: по различным признакам, связанным с заявкой на грант, предсказать, будет ли заявка принята.`
`План решения:`

- Загрузите данные из csv файла. Ознакомьтесь с ними, проверьте наличие пропусков, узнайте типы признаков.

`1. Подготовьте данные к обучению моделей:`

- Отделите целевую переменную Grant.Status и выясните, сбалансированы ли классы. Если классы не сбалансированы, используйте в работе хотя бы один из изученных методов борьбы с дисбалансом классов;

`Заполните пропуски`

- в количественных признаках заполните пропуски средними значениями и нулями (у каждой фичи будет по два варианта),

- в категориальных признаках выведите значения, которые они принимают, и найдите признак, где пропуски можно заполнить осмысленным значением. В остальных признаках заполните пропуски значением по умолчанию;

- Преобразуйте категориальные признаки в количественные с помощью прямого кодирования;

- Разделите данные на обучающую и тестовую части;

- Используйте масштабирование для получения признаков одинакового масштаба.

`2. Обучите модели и выберите лучшую:`

- Обучите модель `логистической` `регрессии`, `используя кросс-валидацию`. `Оцените` ее качество с помощью метрики `rocauc`. `Выведите` `топ-10 признаков` по важности, согласно обученной модели;

- Обучите модель `случайного леса`

`Для подбора `гиперпараметров и кросс-валидации используйте `структуру GridSearchCV`,

- Выберите наилучший вариант случайного леса и выведите его параметры,

- Оцените качество выбранной модели с помощью метрики rocauc,

- Выведите топ-10 признаков по важности. Используйте `атрибут` `feature_importances_`, чтобы узнать важность признаков в деревянных моделях.

Напишите вывод, в котором будет содержаться информация о том, какие признаки важны согласно обеим моделям, и какая модель оказалась наилучшей для решения поставленной задачи.

In [1]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier

In [2]:
df = pd.read_csv('grant_data_imb.csv')
df.shape

(4113, 39)

`1. Подготовьте данные к обучению моделей:`

- Отделите целевую переменную Grant.Status и выясните, сбалансированы ли классы. Если классы не сбалансированы, используйте в работе хотя бы один из изученных методов борьбы с дисбалансом классов;

`Заполните пропуски`

- в количественных признаках заполните пропуски средними значениями и нулями (у каждой фичи будет по два варианта),

- в категориальных признаках выведите значения, которые они принимают, и найдите признак, где пропуски можно заполнить осмысленным значением. В остальных признаках заполните пропуски значением по умолчанию;

- Преобразуйте категориальные признаки в количественные с помощью прямого кодирования;

- Разделите данные на обучающую и тестовую части;

- Используйте масштабирование для получения признаков одинакового масштаба.

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4113 entries, 0 to 4112
Data columns (total 39 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   Grant.Status                            4113 non-null   int64  
 1   Sponsor.Code                            3856 non-null   object 
 2   Grant.Category.Code                     3856 non-null   object 
 3   Contract.Value.Band...see.note.A        1953 non-null   object 
 4   RFCD.Code.1                             3853 non-null   float64
 5   RFCD.Percentage.1                       3853 non-null   float64
 6   RFCD.Code.2                             3853 non-null   float64
 7   RFCD.Percentage.2                       3853 non-null   float64
 8   RFCD.Code.3                             3853 non-null   float64
 9   RFCD.Percentage.3                       3853 non-null   float64
 10  RFCD.Code.4                             3853 non-null   floa

In [4]:
df.describe(include='all').fillna('-').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
Grant.Status,4113.0,-,-,-,0.207634,0.405663,0.0,0.0,0.0,0.0,1.0
Sponsor.Code,3856,226,4D,1006,-,-,-,-,-,-,-
Grant.Category.Code,3856,13,10A,2050,-,-,-,-,-,-,-
Contract.Value.Band...see.note.A,1953,16,A,961,-,-,-,-,-,-,-
RFCD.Code.1,3853.0,-,-,-,314904.682845,47163.318702,210000.0,280401.0,320801.0,321202.0,999999.0
RFCD.Percentage.1,3853.0,-,-,-,74.69686,26.875419,5.0,50.0,80.0,100.0,100.0
RFCD.Code.2,3853.0,-,-,-,161386.717104,161577.090361,0.0,0.0,240202.0,320702.0,440207.0
RFCD.Percentage.2,3853.0,-,-,-,17.642616,19.259007,0.0,0.0,10.0,30.0,90.0
RFCD.Code.3,3853.0,-,-,-,96437.197508,148599.260202,0.0,0.0,0.0,270208.0,440207.0
RFCD.Percentage.3,3853.0,-,-,-,7.089541,11.937533,0.0,0.0,0.0,15.0,70.0


In [5]:
# выводим количество уникальных значений в категориальных столбцах
categorical_columns = df.select_dtypes(include=['object']).columns 

for category in categorical_columns:            
    print("Статистика для столбца -", category)
    print(df[category].value_counts())          
    print(df[category].describe())              
    print(f'{"-"*75}')

Статистика для столбца - Sponsor.Code
Sponsor.Code
4D      1006
2B       915
21A      375
24D      114
40D       91
        ... 
242B       1
308D       1
284D       1
259C       1
225A       1
Name: count, Length: 226, dtype: int64
count     3856
unique     226
top         4D
freq      1006
Name: Sponsor.Code, dtype: object
---------------------------------------------------------------------------
Статистика для столбца - Grant.Category.Code
Grant.Category.Code
10A    2050
30B     707
50A     375
10B     211
20C     180
30C     147
30D      93
20A      49
30G      35
30E       5
30A       2
40C       1
30F       1
Name: count, dtype: int64
count     3856
unique      13
top        10A
freq      2050
Name: Grant.Category.Code, dtype: object
---------------------------------------------------------------------------
Статистика для столбца - Contract.Value.Band...see.note.A
Contract.Value.Band...see.note.A
A     961
B     305
C     159
D     151
G     135
E      98
F      75
H      33
J 

In [6]:
print(f'Количество дубликатов в DataFrame = {df.duplicated().sum()}')

Количество дубликатов в DataFrame = 65


In [7]:
df.drop_duplicates(inplace=True)

In [8]:
print(f'    Количество NaN в DataFrame \n{"-"*50}\n{df.isna().sum()}')

    Количество NaN в DataFrame 
--------------------------------------------------
Grant.Status                                 0
Sponsor.Code                               245
Grant.Category.Code                        245
Contract.Value.Band...see.note.A          2117
RFCD.Code.1                                249
RFCD.Percentage.1                          249
RFCD.Code.2                                249
RFCD.Percentage.2                          249
RFCD.Code.3                                249
RFCD.Percentage.3                          249
RFCD.Code.4                                249
RFCD.Percentage.4                          249
RFCD.Code.5                                249
RFCD.Percentage.5                          249
SEO.Code.1                                 264
SEO.Percentage.1                           264
SEO.Code.2                                 264
SEO.Percentage.2                           264
SEO.Code.3                                 264
SEO.Percentage.3        

Разделим данные на фичи и целевой признак

In [9]:
target = df['Grant.Status']
features = df.drop(['Grant.Status'], axis = 1)
target.value_counts()

Grant.Status
0    3209
1     839
Name: count, dtype: int64

- `Данные в датасете представлены разрозненно. Имеются пропуски, несколько категориальных столбцов. наблюдается сильный дисбаланс классов.`

In [10]:
def upsample(features, target, repeat=10): # repeat - отвечает за то во сколько раз необходимо увеличить объекты более редкого класса 
    # разделяем объекты разных классов и информацию о них по разным переменным
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    # дублируем записи объектов более редкого класса
    if len(target_ones) > len(target_zeros):
        repeat = round(len(target_ones) / len(target_zeros))
        features_upsampled = pd.concat([features_ones] + [features_zeros] * repeat)
        target_upsampled = pd.concat([target_ones] + [target_zeros] * repeat)
    else:
        repeat = round(len(target_zeros) / len(target_ones))
        features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
        target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    
    # перемешиваем объекты
    features_upsampled, target_upsampled = shuffle(features_upsampled, target_upsampled, random_state=23) # перемешиваем 
    
    return features_upsampled, target_upsampled

In [11]:
features_train_upsampled, target_train_upsampled = upsample(features, target)
target_train_upsampled.value_counts()

Grant.Status
1    3356
0    3209
Name: count, dtype: int64

- Классы сбалансированны 

заполняем пропуски средними значениями и нулями (у каждой фичи будет по два варианта)

In [12]:
for column in features.columns:
    if np.issubdtype(features[column].dtype, np.float64): # является ли тип данных столбца float64
        new_column = f'{column}_0'
        features[new_column] = features[column].fillna(0) # создадим столбец new_column с заменой отсутствующих значений на 0
        features[column].fillna(features[column].mean(), inplace=True) # исходном столбце  `column` отсутствующие значения заменяются на среднее

- проанализируем категориальные признаки

In [13]:
for col in features.columns:
    if features[col].dtype == 'object':
        print(col)
        print(features[col].value_counts())
        print("-"*50)

Sponsor.Code
Sponsor.Code
4D      979
2B      913
21A     374
24D     114
40D      89
       ... 
308D      1
284D      1
259C      1
331C      1
225A      1
Name: count, Length: 226, dtype: int64
--------------------------------------------------
Grant.Category.Code
Grant.Category.Code
10A    2020
30B     699
50A     374
10B     207
20C     177
30C     145
30D      91
20A      48
30G      34
30E       5
30A       1
40C       1
30F       1
Name: count, dtype: int64
--------------------------------------------------
Contract.Value.Band...see.note.A
Contract.Value.Band...see.note.A
A     951
B     302
C     156
D     147
G     135
E      97
F      75
H      32
J      18
I      11
P       2
K       1
M       1
O       1
Q       1
L       1
Name: count, dtype: int64
--------------------------------------------------
Role.1
Role.1
CHIEF_INVESTIGATOR         3601
EXT_CHIEF_INVESTIGATOR      212
PRINCIPAL_SUPERVISOR        141
DELEGATED_RESEARCHER         36
STUD_CHIEF_INVESTIGATOR      10
HO

- заполним пропуски в категориальных столбцах. Принято решение заполнить строкой `no_information`

In [14]:
features['With.PHD.1'].fillna('NO', inplace=True)

for col in features.columns:
    if features[col].dtype == 'object':
        features[col].fillna('no_information', inplace=True)

In [15]:
features.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4048 entries, 0 to 4112
Data columns (total 68 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   Sponsor.Code                            4048 non-null   object 
 1   Grant.Category.Code                     4048 non-null   object 
 2   Contract.Value.Band...see.note.A        4048 non-null   object 
 3   RFCD.Code.1                             4048 non-null   float64
 4   RFCD.Percentage.1                       4048 non-null   float64
 5   RFCD.Code.2                             4048 non-null   float64
 6   RFCD.Percentage.2                       4048 non-null   float64
 7   RFCD.Code.3                             4048 non-null   float64
 8   RFCD.Percentage.3                       4048 non-null   float64
 9   RFCD.Code.4                             4048 non-null   float64
 10  RFCD.Percentage.4                       4048 non-null   float64
 

- преобразовываем категориальные переменные с помощью One-Hot Encoding (одномерного кодирования) функция `get_dummies()\
Эта функция создает новые столбцы для каждого уникального значения категориальной переменной и устанавливает значение 1 в соответствующем столбце, если значение присутствует, и 0, если значение отсутствует.

In [16]:
features_ohe = pd.get_dummies(features, drop_first=True) # выкидываем один из сгенерированных столбцов
features_ohe.sample()

,RFCD.Code.1,RFCD.Percentage.1,RFCD.Code.2,RFCD.Percentage.2,RFCD.Code.3,RFCD.Percentage.3,RFCD.Code.4,RFCD.Percentage.4,RFCD.Code.5,RFCD.Percentage.5,...,Country.of.Birth.1_Western Europe,Country.of.Birth.1_no_information,Home.Language.1_Other,Home.Language.1_no_information,With.PHD.1_Yes,No..of.Years.in.Uni.at.Time.of.Grant.1_>5 to 10,No..of.Years.in.Uni.at.Time.of.Grant.1_>=0 to 5,No..of.Years.in.Uni.at.Time.of.Grant.1_Less than 0,No..of.Years.in.Uni.at.Time.of.Grant.1_more than 15,No..of.Years.in.Uni.at.Time.of.Grant.1_no_information
2964,320702.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,False,False,False,True,True,False,False,False,False,False


In [17]:
features_train, features_valid, target_train, target_valid = train_test_split(features_ohe, target, test_size=0.25, random_state=25)

- Для обучения нашей линейной модели мы приводим столбцы к одному масштабу в этом нам поможет StandardScaler

In [18]:
scaler = StandardScaler()

# настраиваем только на обучающей части
scaler.fit(features_train) 

# применяем обученный StandardScaler на обои  части 
features_train_sc = scaler.transform(features_train)
features_valid_sc = scaler.transform(features_valid)

In [19]:
pd.DataFrame(features_train_sc, columns=features_train.columns).sample()

,RFCD.Code.1,RFCD.Percentage.1,RFCD.Code.2,RFCD.Percentage.2,RFCD.Code.3,RFCD.Percentage.3,RFCD.Code.4,RFCD.Percentage.4,RFCD.Code.5,RFCD.Percentage.5,...,Country.of.Birth.1_Western Europe,Country.of.Birth.1_no_information,Home.Language.1_Other,Home.Language.1_no_information,With.PHD.1_Yes,No..of.Years.in.Uni.at.Time.of.Grant.1_>5 to 10,No..of.Years.in.Uni.at.Time.of.Grant.1_>=0 to 5,No..of.Years.in.Uni.at.Time.of.Grant.1_Less than 0,No..of.Years.in.Uni.at.Time.of.Grant.1_more than 15,No..of.Years.in.Uni.at.Time.of.Grant.1_no_information
2227,-0.760878,-1.345645,0.759318,1.19957,1.277656,1.139986,-0.152962,-0.145071,-0.074909,-0.07024,...,-0.195713,-0.430667,-0.137084,0.332357,0.864196,2.126029,-0.705012,-0.373684,-0.314966,-0.466178


- получаем признаки примерно одного масштаба

-------------
`2. Обучите модели и выберите лучшую:`

- Обучите модель `логистической` `регрессии`, `используя кросс-валидацию`. `Оцените` ее качество с помощью метрики `rocauc`. `Выведите` `топ-10 признаков` по важности, согласно обученной модели;

`2.1` Обучите модель `случайного леса`

- `Для подбора `гиперпараметров и кросс-валидации используйте `структуру GridSearchCV`,

- Выберите наилучший вариант случайного леса и выведите его параметры,

- Оцените качество выбранной модели с помощью метрики rocauc,

- Выведите топ-10 признаков по важности. Используйте `атрибут` `feature_importances_`, чтобы узнать важность признаков в деревянных моделях.

In [27]:
model = LogisticRegressionCV(solver='liblinear', random_state=15, class_weight='balanced', cv=12)
model.fit(features_train_sc,target_train)
roc_auc_score(target_valid,model.predict_proba(features_valid_sc)[:,1])

0.8543820267710811

- посмотрим какие Фичи самые важные 

In [23]:
data = pd.DataFrame(zip(list(features_ohe.columns), list(abs(model.coef_[0]))), columns=['features', 'importance'])
sorted_data = data.sort_values(by='importance', ascending=False).head(10)

display(sorted_data)

,features,importance
53,Faculty.No..1_0,1.909341
51,Year.of.Birth.1_0,1.887674
23,Faculty.No..1,1.520222
314,Contract.Value.Band...see.note.A_no_information,1.455511
317,Role.1_EXT_CHIEF_INVESTIGATOR,1.427784
244,Sponsor.Code_4D,1.254921
52,Dept.No..1_0,1.142255
262,Sponsor.Code_6B,1.127367
55,Number.of.Unsuccessful.Grant.1_0,1.032548
199,Sponsor.Code_2B,1.029650


ОБУЧИМ МОДЕЛЬ СЛУЧАЙНОГО ЛЕСА

-- создаем объект классификатора RandomForestClassifier(), задаем набор гиперпараметров
-  `n_estimators` : количество деревьев в лесу (от 10 до 50 с шагом 10)
-  `max_depth` : максимальная глубина деревьев (от 1 до 12 с шагом 2)
-  `min_samples_leaf` : минимальное количество объектов в листе дерева (от 1 до 7)
-  `min_samples_split` : минимальное количество объектов, необходимое для разделения узла (от 2 до 9)

GridSearchCV (поиск по сетке) выполняет перебор всех возможных комбинаций гиперпараметров, заданных в  `parametrs_RFC` , и оценивает производительность модели с помощью кросс-валидации на 10 фолдах (cv=10).

`features_train_sc`  и  `target_train`  представляют собой тренировочные данные, которые будут использоваться для обучения модели.

После выполнения  `grid.fit()` , можно получить наилучшие значения гиперпараметров с помощью атрибута  `best_params_`  объекта  `grid` . Например,  `grid.best_params_`  вернет словарь с наилучшими значениями гиперпараметров, найденными в результате поиска.

In [26]:
clf = RandomForestClassifier()
parametrs_RFC = {'n_estimators': range (10,51,10),
                 'max_depth': range (1,13,2),
                 'min_samples_leaf': range (1,8),
                 'min_samples_split': range (2,10)}

grid = GridSearchCV(clf, parametrs_RFC, cv=10)
grid.fit(features_train_sc,target_train)

GridSearchCV(cv=10, estimator=RandomForestClassifier(),
             param_grid={'max_depth': range(1, 13, 2),
                         'min_samples_leaf': range(1, 8),
                         'min_samples_split': range(2, 10),
                         'n_estimators': range(10, 51, 10)})

отобразим подобранные параметры модели

In [28]:
grid.best_params_

{'max_depth': 11,
 'min_samples_leaf': 1,
 'min_samples_split': 6,
 'n_estimators': 30}

Обучаем модель `RandomForestClassifier`, задаем гиперпараметры, используя функцию fit(). Затем вычислим значение метрики `roc_auc_score` для `валидационных` данных, используя функцию predict_log_proba() для получения предсказанных `вероятностей` и выбирая второй столбец с помощью [:,1].

In [43]:
model = RandomForestClassifier(random_state=12, class_weight='balanced', max_depth=11, min_samples_leaf=3, min_samples_split=6, n_estimators=41)
model.fit(features_train_sc,target_train)
roc_auc_score(target_valid, model.predict_log_proba(features_valid_sc)[:,1])

0.913395743921026

Отразим Фичи и их значение при обучении модели RandomForestClassifier

Для этого:

1. Создам DataFrame. В этом DataFrame каждая строка содержит пару значений: название признака (features_ohe.columns) и его важность (model.feature_importances_).
2. Задам названия столбцов DataFrame с помощью параметра  `columns`.
3. Отсортирую DataFrame по столбцу 'importance' в порядке убывания с помощью функции `sort_values(by=['importance'],ascending=False)`.
4. Из отсортированного DataFrame выбираю первых 15 строк.

In [42]:
pd.DataFrame(zip(list(features_ohe.columns), model.feature_importances_),
             columns = ['features', 'importance']).sort_values(by=['importance'],ascending=False).head(15)

,features,importance
314,Contract.Value.Band...see.note.A_no_information,0.173428
25,Number.of.Unsuccessful.Grant.1,0.122061
55,Number.of.Unsuccessful.Grant.1_0,0.077376
54,Number.of.Successful.Grant.1_0,0.042580
24,Number.of.Successful.Grant.1,0.034511
285,Sponsor.Code_no_information,0.025847
298,Grant.Category.Code_no_information,0.024798
30,RFCD.Code.1_0,0.017424
40,SEO.Code.1_0,0.015053
0,RFCD.Code.1,0.014424


`ВЫВОДЫ:`

Модель `LogisticRegressionCV` - `roc_auc_score` = 0.`85438`

--- Фичи по важности ---

- 53	Faculty.No..1_0	                                - 1.909341
- 51	Year.of.Birth.1_0	                            - 1.887674
- 23	Faculty.No..1	                                - 1.520222
- 314	Contract.Value.Band...see.note.A_no_information	- 1.455511


Модель `RandomForestClassifier` - `roc_auc_score` = 0.`91339`

--- Фичи по важности ---

- 314	Contract.Value.Band...see.note.A_no_information	- 0.173428
- 25	Number.of.Unsuccessful.Grant.1	                - 0.122061
- 55	Number.of.Unsuccessful.Grant.1_0	            - 0.077376
- 54	Number.of.Successful.Grant.1_0	                - 0.042580

> При подготовке данных для построения моделей приходится очень плотно работать с датасетом, погружаться в данные, разбираться в логических связях.
При обучении двух моделей видна существенная разница в результате (согласно метрики roc_auc_score). Интересным, так же является потраченное время на подбор гиперпараметров
модели RandomForestClassifier при помощи GridSearchCV (у меня было два результата 27 и 47 минут), так же при просмотре первых 4х самых важных фичей у каждой модели можно заметить сходство, только по одной фиче - `314	Contract.Value.Band...see.note.A_no_information`